In [6]:
##Setting the enviromment: 

#1# Useful: 
import pandas as pd  #panda library
import numpy as np   #numpy library
import missingno as msno # data cleaning
import psycopg2   # 
from sqlalchemy import create_engine  #
import os
from dotenv import load_dotenv    # 

#2# Data visualization: 
import matplotlib.pyplot as plt
import seaborn as sns

In [7]:
## Setting the data frame: 
load_dotenv()

DATABASE = os.getenv('DATABASE')
USER_DB = os.getenv('USER_DB')
PASSWORD = os.getenv('PASSWORD')
HOST = os.getenv('HOST')
PORT = os.getenv('PORT')

conn = psycopg2.connect(
    database=DATABASE,
    user=USER_DB,
    password=PASSWORD,
    host=HOST,
    port=PORT
)
cur = conn.cursor()
query_string = "SELECT ks.price , kd.zipcode, kd.grade, kd.condition, ks.house_id, kd.sqft_living, kd.sqft_lot15,ks.house_id  FROM eda.king_county_house_details kd FULL JOIN eda.king_county_house_sales ks ON ks.house_id  = kd.id "
df_nicole = pd.read_sql(query_string, conn)
df_nicole.to_csv('data/eda.csv',index=False)
conn.close()

C:\Users\Giu\AppData\Local\Temp\ipykernel_11524\2938442450.py:19: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_nicole = pd.read_sql(query_string, conn)


In [8]:
##Cleaning and organizing the data: 

df_nicole = pd.read_csv('data/eda.csv')
#df_nicole.duplicated().value_counts()  --> first check on duplicates 
#df_nicole = df_task_01.drop_duplicates()
df_nicole.columns.drop_duplicates()
df_nicole.drop(df_nicole.columns[[-1]], axis=1, inplace=True)
df_nicole.reset_index(inplace=True, drop=True)

df_nicole.fillna(0, inplace=True)  ##--> making sure I won't have a NaN

df_nicole.columns  ##-->checking the colummn names for the next questions

#df_task_01.head()

Index(['price', 'zipcode', 'grade', 'condition', 'house_id', 'sqft_living',
       'sqft_lot15'],
      dtype='object')